# <span style="color: green; font-size: 40px; font-weight: bold;"> Projeto 2 (Classificação) </span>

<br><br>

# Classificação de Fraude em Transações Bancárias

<br><br>

### Contexto

- Um banco está preocupado com o aumento de fraudes em transações bancárias e quer usar um modelo de classificação para identificar transações fraudulentas.
- O **objetivo** é proteger os clientes e minimizar as perdas financeiras devido a fraudes.

<br>

### Problema de Negócio
- Como podemos identificar se uma transação bancária é fraudulenta ou não com base em suas características?

<br>

### Sobre o Conjunto de Dados

O conjunto de dados simulado terá 10.000 linhas e incluirá as seguintes colunas:

<br>

<table border="2">
  <tr>
    <th style="text-align: center; font-size: 16px;">Nome da Coluna</th>
    <th style="text-align: center; font-size: 16px;">Tipo de Dado</th>
    <th style="text-align: center; font-size: 16px;">Descrição</th>
  </tr>
  <tr>
    <td>transaction_id</td>
    <td>str</td>
    <td>Identificação única da transação.</td>
  </tr>
  <tr>
    <td>account_age</td>
    <td>int</td>
    <td>Idade da conta em meses.</td>
  </tr>
  <tr>
    <td>transaction_amount</td>
    <td>float</td>
    <td>Valor da transação em reais.</td>
  </tr>
  <tr>
    <td>transaction_type</td>
    <td>str</td>
    <td>Tipo de transação (compra, transferência, saque).</td>
  </tr>
  <tr>
    <td>location</td>
    <td>str</td>
    <td>Localização da transação.</td>
  </tr>
  <tr>
    <td>is_fraud</td>
    <td>int</td>
    <td>Indicador se a transação é fraudulenta (1) ou não (0).</td>
  </tr>
</table>

<br><br>

### Perguntas de Negócio Adicionais:

- Quais padrões de localização estão associados a transações fraudulentas?

> Analisando a localização das transações, podemos identificar se fraudes ocorrem mais frequentemente em determinadas áreas geográficas. Isso pode ajudar a detectar atividades suspeitas em regiões específicas e implementar medidas de segurança mais eficazes.

- Como o valor da transação e o tipo de transação influenciam a probabilidade de fraude?

> Investigando a relação entre o valor da transação e seu tipo (compra, transferência, saque), podemos identificar combinações que são mais susceptíveis a fraudes. Isso permitirá ao banco criar alertas específicos para transações de alto risco.

<br><br>

## Importando Pacotes

In [1]:
# Manipulação de Dados
import numpy as np
import pandas as pd

# Criação e Visualização de Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# Carrega a função SMOTE (Balanceamento de Classes)
import imblearn
from imblearn.over_sampling import SMOTE

# Pacote do Python para Machine Learning
import sklearn

# Função model_selection do pacote sklearn
from sklearn.model_selection import train_test_split    # dividir os dados em treino e teste
from sklearn.model_selection import GridSearchCV        # técnica de utilização de hiperparâmetros
from sklearn.model_selection import RandomizedSearchCV  # técnica de utilização de hiperparâmetros
from sklearn.model_selection import cross_val_score     # validação cruzada (para avaliação do modelo)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


## Pacotes do sklearn com os Algoritmos

# Função com Algoritmo de ML (Regressão Logística)
from sklearn.linear_model import LogisticRegression     

# Função com Algoritmo de ML (RandomForest)
from sklearn.ensemble import RandomForestClassifier

# Função com Algoritmo de ML (KNN)
from sklearn.neighbors import KNeighborsClassifier

# Função com Algoritmo de ML (Árvores de Decisão)
from sklearn.tree import DecisionTreeClassifier

# Função com Algoritmo de ML (SVC)
from sklearn.svm import SVC

# Calcular as Métricas para Avaliação dos Modelos
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix
from sklearn.metrics import accuracy_score

# Salvar o modelo após o treinamento
import joblib
import pickle

# Indicando que os Gráficos apareçam no Jupyter Notebook
%matplotlib inline

# Mensagens de Aviso
import warnings
warnings.filterwarnings("ignore")

<br><br>

## Carregando Conjunto de Dados

In [5]:
# Carrega o dataset
dados = pd.read_csv('dados/transacoes_bancarias.csv')

In [6]:
# Visualiza
dados.head()

,transaction_id,account_age,transaction_amount,transaction_type,location,is_fraud
0,TX0,103.0,154.239105,saque,online,0
1,TX1,180.0,2.859674,saque,online,0
2,TX2,93.0,59.582040,saque,online,0
3,TX3,15.0,119.546919,transferência,online,0
4,TX4,107.0,236.122174,saque,online,0


In [7]:
# Info
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10100 entries, 0 to 10099
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   transaction_id      10100 non-null  object 
 1   account_age         9998 non-null   float64
 2   transaction_amount  9996 non-null   float64
 3   transaction_type    10100 non-null  object 
 4   location            10100 non-null  object 
 5   is_fraud            10100 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 473.6+ KB
